In [3]:
import json
datasets =None
with open ('datasets5.json', 'r') as f:
    datasets = json.load(f)
min_len = 10000
for i , dataset in enumerate(datasets):
    if dataset['filename'] == '5_data.txt':
        print(i)

for i in range(4, len(datasets)):
    print(datasets[i]['filename'])
    break

4
5_data.txt


In [7]:
import json
import numpy as np
datasets =None
with open ('datasets6.json', 'r') as f:
    datasets = json.load(f)
for dataset in datasets:
    node_token_list = dataset['node_token_list']
    relation_matrix = dataset['relation_matrix']
    print(dataset['filename'])
    print(np.array(relation_matrix))
    

1_data.txt
[[0 0 1 1 1 1 1 0 1 0 0 0 0 1 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 1 0]
 [1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
 [1 0 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 1 0 0]
 [1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 0]
 [0 0 1 0 0 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0]
 [1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 0]
 [0 1 1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1]
 [1 0 1 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 1 0]
 [1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0]
 [1 0 0 1 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0]]
2_data.txt
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [4]:
import json
from graphviz import Digraph
def __generate_edge_index(data_2_mask_single_signal_llm:list[str]) -> list[list[int, int]]:
    '''
        生成邻接矩阵
        data_2_mask_single_signal_llm: list[str], 数据集数据
    '''
    cur_tokens = data_2_mask_single_signal_llm
    cur_no_activity = []
    last_activity = -1
    cur_adj_lists = []
    for j in range(0, len(cur_tokens)):
        if cur_tokens[j] != '[activity]':
            cur_no_activity.append(j)
            if last_activity != -1:
                cur_adj_lists.append([last_activity, j])
                cur_adj_lists.append([j, last_activity])
                # cur_adj_lists[last_activity].add(j)
                # cur_adj_lists[j].add(last_activity)
        elif cur_tokens[j] == '[activity]' and len(cur_no_activity) != 0:
            for k in cur_no_activity:
                cur_adj_lists.append([k, j])
                cur_adj_lists.append([j, k])
                # cur_adj_lists[k].add(j)
                # cur_adj_lists[j].add(k)
            last_activity = j
            cur_no_activity = []
        elif cur_tokens[j] == '[activity]' and len(cur_no_activity) == 0:
            #两个activity直接相连的时候，或许可以考虑在中间加个什么标识
            if last_activity != -1:
                cur_adj_lists.append([last_activity, j])
                cur_adj_lists.append([j, last_activity])
                # cur_adj_lists[last_activity].add(j)
                # cur_adj_lists[j].add(last_activity)
            last_activity = j
    cur_adj_lists = sorted(cur_adj_lists, key=lambda x: x[1])
    return cur_adj_lists

datasets =None
with open ('datasets6.json', 'r') as f:
    datasets = json.load(f)
for dataset in datasets:
    data_2_mask_single_signal_llm = dataset['data_2_mask_single_signal_llm']
    node_token_list = dataset['node_token_list']
    filename = dataset['filename']
    edge_index = __generate_edge_index(data_2_mask_single_signal_llm)
    dot = Digraph(name=filename, format='png')
    for i, node in enumerate(node_token_list):
        dot.node(str(i), node)
    for edge in edge_index:
        dot.edge(str(edge[0]), str(edge[1]))
    dot.render(filename, directory='/home/btr/bpmn/LLMEnG/my-data/image')
    break